<a href="https://colab.research.google.com/github/chihchao/application-of-programming/blob/main/web_api_linebot_template_message.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

In [7]:
from flask import Flask, request, abort
# for  colab
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
from google.colab import userdata
# for line bot
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, TemplateSendMessage, ConfirmTemplate, MessageAction,
    ButtonsTemplate, URIAction, CameraAction, CameraRollAction, LocationAction,
    CarouselTemplate, CarouselColumn,
    ImageCarouselTemplate, ImageCarouselColumn,
    PostbackAction, PostbackEvent
)

def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    public_url = ngrok.connect(5000)
    run_with_ngrok(app)

app = Flask(__name__)
ngrok_start()

line_bot_api = LineBotApi(userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_text(event):
    # confirm template
    if event.message.text == 'confirm':
        reply = TemplateSendMessage(
            alt_text='這是一個確認訊息',
            template=ConfirmTemplate(
                text='你喜歡芙莉蓮嗎？',
                actions=[
                    MessageAction(label='是', text='我超喜歡的'),
                    MessageAction(label='否', text='雖然很喜歡，但我要傲驕的說否')
                ]
            )
        )
    # buttons template
    elif event.message.text == 'buttons':
        reply = TemplateSendMessage(
            alt_text='這是一個按鈕選單',
            template=ButtonsTemplate(
                thumbnail_image_url='https://cdn.pixabay.com/photo/2018/10/20/09/28/budapest-3760434_640.jpg',
                title='功能選單',
                text='請選擇一項操作',
                actions=[
                    URIAction(label='Pixabay', uri='https://pixabay.com/'),
                    CameraAction(label='拍照'),
                    CameraRollAction(label='選擇相片'),
                    LocationAction(label='選擇位置')
                ]
            )
        )
    elif event.message.text == 'carousel':
        reply = TemplateSendMessage(
            alt_text='這是選單列表',
            template=CarouselTemplate(
                columns=[
                    CarouselColumn(
                        thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/7/7d/Summer_Wars_poster.jpg',
                        title='夏日大作戰',
                        text='2009年上映的日本科幻暨浪漫電影',
                        actions=[
                            MessageAction(label='選我', text='細田守一票'),
                            URIAction(label='看更多', uri='https://zh.wikipedia.org/zh-tw/%E5%A4%8F%E6%97%A5%E5%A4%A7%E4%BD%9C%E6%88%B0')
                        ]
                    ),
                    CarouselColumn(
                        thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/thumb/4/4f/Castle_of_Cagliostro_poster.png/250px-Castle_of_Cagliostro_poster.png',
                        title='魯邦三世 卡里奧斯特羅城',
                        text='1979年上映的日本動畫動作冒險喜劇電影',
                        actions=[
                            MessageAction(label='選我', text='宮崎駿一票'),
                            URIAction(label='看更多', uri='https://zh.wikipedia.org/zh-tw/%E9%AD%AF%E9%82%A6%E4%B8%89%E4%B8%96_%E5%8D%A1%E9%87%8C%E5%A5%A7%E6%96%AF%E7%89%B9%E7%BE%85%E4%B9%8B%E5%9F%8E')
                        ]
                    )
                ]
            )
        )
    elif event.message.text == 'imageCarousel':
        reply = TemplateSendMessage(
            alt_text='這是圖片選單',
            template=ImageCarouselTemplate(
                columns=[
                    ImageCarouselColumn(
                        image_url='https://upload.wikimedia.org/wikipedia/zh/7/7d/Summer_Wars_poster.jpg',
                        action=PostbackAction(
                            label='夏日大作戰',
                            data='action=select&item=A'
                        )
                    ),
                    ImageCarouselColumn(
                        image_url='https://upload.wikimedia.org/wikipedia/zh/thumb/4/4f/Castle_of_Cagliostro_poster.png/250px-Castle_of_Cagliostro_poster.png',
                        action=PostbackAction(
                            label='魯邦三世 卡里奧斯特羅城',
                            data='action=select&item=B'
                        )
                    )
                ]
            )
        )
    else:
        reply = TextSendMessage(text=event.message.text)
    line_bot_api.reply_message(event.reply_token, reply)
# 處理 image_carousel 的 postback 訊息
@handler.add(PostbackEvent)
def handle_postback(event):
    data = event.postback.data

    if data == "action=select&item=A":
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="你選了 夏日大作戰"))
    elif data == "action=select&item=B":
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text="你選了 魯邦三世 卡里奧斯特羅城"))
    else:
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text=f"你選了：{data}"))


# 啟動 web server
app.run()

<ipython-input-7-2abc0bff8124>:29: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))


 * Serving Flask app '__main__'
 * Debug mode: off


<ipython-input-7-2abc0bff8124>:30: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://eff9-104-199-190-212.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [19/Apr/2025 12:06:52] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2025 12:06:53] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2025 12:07:17] "POST /callback HTTP/1.1" 200 -
<ipython-input-7-2abc0bff8124>:123: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(event.reply_token, reply)
INFO:werkzeug:127.0.0.1 - - [19/Apr/2025 12:07:37] "POST /callback HTTP/1.1" 200 -
<ipython-input-7-2abc0bff8124>:132: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more